In [1]:
# for classical solver (simulated annealing)
import dimod
import operator
import dwavebinarycsp
import numpy as np
import math

In [14]:
rows = 'ABCD'
cols = '1234'
boxes = [[("{}{}".format(r, c)) for c in cols] for r in rows]  # declare variables for each box in the puzzle
square_units = [ [ x+y for x in A for y in B ] for A in ('ABCD') for B in ('1234') ]

In [40]:
def print_board(board):
    if isinstance(board, str):     
        board = string_to_tuple(board)
    for row, _boxes in enumerate(boxes):
        if row and row % 2 == 0:
            print('-'*6+"|"+'-'*6)
        for col, box in enumerate(_boxes):
            if col and col % 2 == 0:
                print('|', end='')
            print(' {} '.format((int(board[row][col]) or '-')), end='')
        print()
    print()

In [44]:
board = ((1,2,0,0),
         (0,0,0,0),
         (0,0,1,0),
         (0,0,4,0),
)
print_board(board)

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [47]:
correct_board = ((1,2,3,4),
         (4,3,2,1),
         (2,1,4,3),
         (3,4,1,2),
)
print_board(correct_board)

 1  2 | 3  4 
 4  3 | 2  1 
------|------
 2  1 | 4  3 
 3  4 | 1  2 



In [48]:
from itertools import product

def check_sudoku(grid):
    """Validate a sudoku solution.

    Given a grid as a list of lists, return None if it is ill-formed,
    False if it is invalid, or True if it is a valid solution.
    """
    error_count=0
    # Check that the grid is 4x4.
    if len(grid) != 4 or not all(len(row) == 4 for row in grid):
        error_count+=1
    DIGITS = set(range(1, 5))

    # Check that each number appears exactly once per row
    if not all(set(row) == DIGITS for row in grid):
        error_count+=1

    # Check that each number appears exactly once per column
    columns = [[row[c] for row in grid] for c in range(4)]
    if not all(set(col) == DIGITS for col in columns):
        error_count+=1

    # Check that each number appears exactly once per 3x3 grid
    THREES = [(0, 1), (2, 3)]
    for row_block, col_block in product(THREES, THREES):
        block = [grid[r][c] for r, c in product(row_block, col_block)]
        if set(block) != DIGITS:
            error_count+=1


    return error_count


In [52]:
# check if the solution check works
print('Error Count: ',check_sudoku(correct_board))
print('Error Count: ',check_sudoku(board))

Error Count:  0
Error Count:  6


In [53]:
def encode_board_to_binary(board):
    binary_board= np.zeros((4, 4, 4)).tolist()
    for row_index, row in enumerate(board):
        for column_index, cell in enumerate(row):
            if cell>0:
                binary_board[cell-1][row_index][column_index]=1    
    return binary_board 

def decode_board_from_binary(binary_board):
    board= np.zeros((4, 4))
    for k, color in enumerate(binary_board):
        for i, row in enumerate(color):
            for j, cell_value in enumerate(row):
                if cell_value>0:
                    board[i][j]+=int(k+1)
                else:
                    board[i][j]+=int(0)
    return board 

In [60]:
constant = 0
N = 4
cell_qubo = {}
linear = {}
quad = {}
penalty_weight = 1
for i in range(1,N+1):
        for j in range(1,N+1):
            for k1 in range(1,N+1):  
                    var_1 = encode_var_labels(i,j,k1)
                    for k2 in range(1,N+1):
                            var_2 = encode_var_labels(i,j,k2)
                            if var_1 == var_2:
                                linear[var_1] = -1*penalty_weight
                            else:
                                quad[var_1,var_2]= 2*penalty_weight
                        #linear[var_1] = 2
                    
            constant+=1
            

cell_qubo[()] = constant*penalty_weight
cell_qubo["linear"] = linear
cell_qubo["quadratic"] = quad

#bqm = dimod.BinaryQuadraticModel(cell_qubo,constant,dimod.Vartype.BINARY)

In [61]:
penalty_weight=1
constant = 0
N = 4
column_qubo = {}
lin_column={}
quad_column={}
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for j in range(1,N):
            for i1 in range(1,N):
                var_1 = encode_var_labels(i1,j,k)
                for i2 in range(1,N):               
                    var_2 = encode_var_labels(i2,j,k)
                    if var_1 == var_2:
                        lin_column[var_1] = -1*penalty_weight
                    else:
                        quad_column[var_1,var_2] = 2*penalty_weight
        constant+=1
        
column_qubo[()] = constant*penalty_weight
column_qubo['linear'] = lin_column
column_qubo['quadratic'] = quad_column

In [62]:
constant = 0
N = 4
row_qubo = {}
linear_row ={}
quadratic_row={}
penalty_weight=2
binary_board = encode_board_to_binary(board)
for k in range(1,N):
        for i in range(1,N):
            for j1 in range(1,N):
                var_1 = encode_var_labels(i,j1,k)
                for j2 in range(1,N):      
                    var_2 = encode_var_labels(i,j2,k)
                    if var_1 == var_2:
                        linear_row[var_1] = -1*penalty_weight
                    else:
                        quadratic_row[var_1, var_2] = 2*penalty_weight
        constant+=1
        
row_qubo[()] = constant*penalty_weight
row_qubo['linear'] = linear_row
row_qubo['quadratic'] = quadratic_row

In [63]:
binary_board = encode_board_to_binary(board)
hint = []
penalty_weight = 4

for k, color in enumerate(binary_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
hint_qubo = {}
constant=0
for (i, j, k) in hint:
    re_label = encode_var_labels(i,j,k) 
    hint_qubo[re_label] = -1*penalty_weight
    constant += 1

hint_qubo[()] = constant*penalty_weight


In [130]:
from math import sqrt
constant = 0
N = 4
duplicate_qubo = {}
dupli_linear = {}
dupli_quadr = {}
sqrtN = int(sqrt(N))
for grid_i in range(sqrtN):
    for grid_j in range(sqrtN):
        for k in range(N):
            # there can be only one k in the same subgrid.
            for i1 in range(grid_i * 2, grid_i * 2 + 2):
                for j1 in range(grid_j * 2, grid_j * 2 + 2):
                    for i2 in range(grid_i * 2, grid_i * 2 + 2):
                        var_1 = encode_var_labels(i1+1,j1+1,k+1)
                        for j2 in range(grid_j * 2, grid_j * 2 + 2):
                            var_2 = encode_var_labels(i2+1,j2+1,k+1)
                            if var_1 == var_2:
                                dupli_linear[var_1] = -1
                            else:
                                dupli_quadr[var_1,var_2] = 1
            constant+=1

            
duplicate_qubo[()] = constant
duplicate_qubo['linear'] = dupli_linear
duplicate_qubo['quadratic'] = dupli_quadr

In [131]:
result_qubo_linear = {}
result_qubo_quadratic = {}

In [132]:
for index, value in hint_qubo.items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [133]:
for index, value in cell_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [134]:
for index, value in cell_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [135]:
for index, value in column_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
    else:
        result_qubo_linear[index] = value

In [136]:
for index, value in column_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
    else:
        result_qubo_quadratic[index] = value

In [137]:
for index, value in row_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [138]:
for index, value in row_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [139]:
for index, value in duplicate_qubo['linear'].items():
    if result_qubo_linear.get(index):
        result_qubo_linear[index] += value
        
    else:
        result_qubo_linear[index] = value

In [140]:
for index, value in duplicate_qubo['quadratic'].items():
    if result_qubo_quadratic.get(index):
        result_qubo_quadratic[index] += value
        
    else:
        result_qubo_quadratic[index] = value

In [141]:
constant/2

8.0

In [142]:
constant= cell_qubo.get((),0) + hint_qubo.get((),0) + row_qubo.get((),0) + column_qubo.get((),0) + duplicate_qubo.get((),0)
bqm = dimod.BinaryQuadraticModel(result_qubo_linear,result_qubo_quadratic, constant,dimod.Vartype.BINARY)

In [143]:
# test run
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_set=sampler.sample(bqm, num_reads=30,num_sweeps=2222)


In [144]:
from random import randrange
import time

def find_optimal_solution(best_solution_global, current_solution, current_energy):
    best_solution = {}
    error_count=20
    iteration = 0
    sampler = neal.SimulatedAnnealingSampler()
    start_time = time.perf_counter()
    while error_count>0:
        """
        sample_set = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                                        num_reads=10, num_sweeps=20000,
                                       beta_schedule_type='geometric')
        """
        random_seed = randrange(9000)
        #random_seed = 5832
        num_reads = randrange(12000)
        num_sweeps = randrange(60000)
        num_reads = 5
        num_sweeps=50
        #num_sweeps*=iteration
        pre_anneal = time.perf_counter()
        sample_set=sampler.sample(bqm,seed=random_seed ,num_reads=num_reads,num_sweeps=num_sweeps)
        iteration+=1
        post_anneal = time.perf_counter()
        
        annealing_time = post_anneal-pre_anneal
        for solution, energy in sample_set.data(['sample', 'energy']):
            binary_solution_board= np.zeros((4, 4, 4))
            for index, value in solution.items():
                if type(index) is int and index>0:
                    board_index = decode_var_labels(index)
                    binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
            solution_board=decode_board_from_binary(binary_solution_board)
            error_count_temp = check_sudoku(solution_board)
            
            
            
            current_solution.append(1)
            current_energy.append(energy)
            current_solution[:] = []
            current_energy[:] = []
            current_solution.append(solution)
            current_energy.append(energy)
            
            
        overall_time = time.perf_counter()-start_time
        if error_count_temp<error_count:
            best_solution = solution_board
            best_solution_global.append(solution)
            error_count=error_count_temp
            print("\nError Count:",error_count, "iteration:", iteration, "Energy:", energy,
                  "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps, 
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            
        if iteration%100 == 0:
            print("\nCurrent State:",
                  "Error Count:",error_count_temp, "iteration:", iteration, "Energy:", energy,
                 "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps,
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            

        

In [145]:
import multiprocessing
manager = multiprocessing.Manager()
best_solution = manager.list()
current_solution = manager.list()
current_energy = manager.list()


process = multiprocessing.Process(target=find_optimal_solution, 
            
                                  args= (best_solution, current_solution, current_energy))
process.start()




Error Count: 3 iteration: 1 Energy: -9.0 Seed: 8961 num_reads: 5 sweeps: 50 Annealing Time: 0.012784035003278404 Overall Time: 0.020402085996465757

Error Count: 2 iteration: 2 Energy: -8.0 Seed: 3098 num_reads: 5 sweeps: 50 Annealing Time: 0.005331012012902647 Overall Time: 0.06920660199830309

Error Count: 1 iteration: 8 Energy: -11.0 Seed: 2670 num_reads: 5 sweeps: 50 Annealing Time: 0.004388147994177416 Overall Time: 0.1969865029968787

Error Count: 0 iteration: 52 Energy: -4.0 Seed: 4043 num_reads: 5 sweeps: 50 Annealing Time: 0.004445525992196053 Overall Time: 0.6978367549891118


In [146]:
import time
time.sleep(10)
#wait for terminating in order to have  
#results when doing a (quick) autorun

process.terminate()
process

<Process name='Process-10' pid=1586124 parent=1561924 stopped exitcode=0>

In [147]:
#sample_set = list(current_solution)
sample_set = list(best_solution)

In [148]:
solution={}
solution_2={}
solution_3={}
for number, solution in enumerate(sample_set):
    if number==0:
        solution = solution
    elif number==1:
        solution_2 = solution
    elif number==2:
        solution_3 = solution
        
    
last_solution = solution

In [149]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value


In [150]:
solution_board=decode_board_from_binary(binary_solution_board)
solution_board

array([[4., 2., 1., 3.],
       [3., 1., 2., 4.],
       [1., 4., 3., 2.],
       [2., 3., 4., 1.]])

In [151]:
print_board(board)

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [152]:
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))
print("BQM Energy:", bqm.energy(solution))

 4  2 | 1  3 
 3  1 | 2  4 
------|------
 1  4 | 3  2 
 2  3 | 4  1 

Error Count: 0
BQM Energy: -4.0


In [181]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution_2.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 4  2 | 1  3 
 3  1 | 2  4 
------|------
 1  4 | 3  2 
 2  3 | 4  1 

Error Count: 0


In [154]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_3.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 4  2 | 1  3 
 3  1 | 2  4 
------|------
 1  4 | 3  2 
 2  3 | 4  1 

Error Count: 0


In [155]:
bqm.energy(solution)

-4.0

In [157]:
from dwave.system import DWaveSampler, EmbeddingComposite
# Use a D-Wave system as the sampler
sampler = DWaveSampler() 

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [158]:
# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(sampler)


In [159]:
bqm_graph = bqm.to_networkx_graph()

In [161]:
print('Number of problem nodes (variables):',len(bqm_graph.nodes))
print('Number of problem edges (couplings):',len(bqm_graph.edges))

Number of problem nodes (variables): 65
Number of problem edges (couplings): 228


In [162]:
from dimod import BinaryQuadraticModel
from dwave.embedding import embed_bqm, unembed_sampleset
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.embedding.chain_breaks import majority_vote

solver = DWaveSampler()

__, target_edgelist, target_adjacency = solver.structure

#bqm = BinaryQuadraticModel.from_qubo(Q)

emb = find_embedding(bqm.to_qubo()[0], target_edgelist)

#embedded_bqm = embed_bqm(bqm, emb, target_adjacency)

#result = solver.sample(embedded_bqm, num_reads=1)

#unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote(bqm, emb))

In [164]:
embedded_bqm = embed_bqm(source_bqm=bqm,embedding=emb ,target_adjacency=target_adjacency)

In [165]:
result = solver.sample(embedded_bqm, num_reads=1)

In [168]:
unembedded = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote)

In [190]:
unembedded

SampleSet(rec.array([([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 21., 1)],
          dtype=[('sample', 'i1', (65,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [1, 41, 6, 60, (), 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64], {'timing': {'qpu_sampling_time': 104, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 63, 'qpu_access_time': 26160, 'qpu_access_overhead_time': 14934, 'qpu_programming_time': 26056, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 454, 'post_processing_overhead_time': 454}, 'problem_id': 'b6aee190-a5fb-4b73-b771-ca569faf6ab2'}, 'BINARY')

In [202]:
d_wave_solution= unembedded.first.sample

In [204]:
import dwave.inspector
dwave.inspector.show(result)   

ValueError: invalid combination of arguments provided: if data capture not enabled, problem/response/solver/etc have to be explicitly specified

In [205]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in d_wave_solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 4  2 | 1  3 
 3  1 | 2  4 
------|------
 1  4 | 3  2 
 2  3 | 4  1 

Error Count: 0
